In [36]:
# %pip install requests 

In [37]:
import requests
from bs4 import BeautifulSoup

def content_scraper(soup, links, identifier):
    link_and_data = {}

    for link in links:
        response = requests.get(links[0])

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            match identifier:
                case 'jena':
                    title = soup.find_all('title')
                    content = soup.find_all('div', class_='flex-layout')

                    text_content = title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                case 'siegburg':
                    divs = soup.find_all('div', class_='flex-layout')
                    content = divs[1]

                    text_content = content.get_text(strip=True)
                case _:
                    text_content = "Wrong identifier"

            link_and_data[link] = text_content

        else:
            message = "Failed to retrieve the webpage. Status code:", response.status_code
            link_and_data[link] = message
    
    return link_and_data

In [38]:
def siegburg_data(soup):
    links = []

    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'siegburg')

    return link_and_content


def jena_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resource-item--title')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.jena.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'jena')

    return link_and_content

In [39]:
urls = [
        'https://mitmachen.siegburg.de/angebotslandkarte',
        'https://mitmachen.jena.de/projekts'
    ]


def def_42(urls):
    urls_and_data = {}

    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve the webpage. Status code:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        link_and_content = scraper(soup, url)
        urls_and_data[url] = link_and_content

    return urls_and_data


def scraper(soup, url):
    if 'siegburg' in url:
        return siegburg_data(soup)
    elif 'jena' in url:
        return jena_data(soup)
    
    return "No scraper defined for this URL"


link_and_data = def_42(urls)

In [40]:
link_and_data

{'https://mitmachen.siegburg.de/angebotslandkarte': {'https://mitmachen.siegburg.de/proposals/1090-senioiren-cafe-lichtblick-kaldauen': 'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*inne

In [41]:
import pandas as pd

# Separate URLs and content for Siegburg and Jena
siegburg_data = {k: v for k, v in link_and_data.items() if "siegburg" in k}
jena_data = {k: v for k, v in link_and_data.items() if "jena" in k}

# Create DataFrames
df_sieburg = pd.DataFrame(list(siegburg_data.items()), columns=["URL", "Content"])
df_jena = pd.DataFrame(list(jena_data.items()), columns=["URL", "Content"])


In [ ]:
df_jena = pd.DataFrame(df_jena['Content'][0].items(), columns=["URL", "Content"])

In [45]:
df_sieburg = pd.DataFrame(df_sieburg['Content'][0].items(), columns=["URL", "Content"])

In [70]:
import re

# Enhanced function to extract all logical parts, including "Unterstützer*innen"
def extract_full_data_with_supporters(content):
    # Extract title (everything before the first date)
    title_match = re.search(r'^(.*?)(\r|\d{1,2}\.\s\w+\s\d{4})', content)
    title = title_match.group(1).strip() if title_match else None

    # Extract date
    date_match = re.search(r'\d{1,2}\.\s\w+\s\d{4}', content)
    date = date_match.group(0) if date_match else None

    # Extract comments count
    comments_match = re.search(r'(\d+)\sKommentare', content)
    comments = int(comments_match.group(1)) if comments_match else 0

    # Extract tags (sections with numbers or + signs)
    tags_match = re.findall(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    tags = ', '.join(tags_match) if tags_match else None

    # Extract description (everything after "Geselliges Beisammensein" or similar patterns)
    description_start = re.search(r'(Geselliges Beisammensein|Angebotslandkarte)', content)
    description = content[description_start.start():].strip() if description_start else None

    # Extract username
    username_match = re.search(r'(\w+\s\w+|Beigetreten am:.*?\d{4})', content)
    username = username_match.group(1).split('Beigetreten am:')[0].strip() if username_match else None

    # Extract Vorschläge count
    vorschlaege_match = re.search(r'Vorschläge(\d+)', content)
    vorschlaege = int(vorschlaege_match.group(1)) if vorschlaege_match else 0

    # Extract Konto verification status
    konto_match = re.search(r'(Konto\s(verifiziert|ist nicht verifiziert))', content)
    konto_status = konto_match.group(2) if konto_match else None

    # # Extract registration date
    # registration_match = re.search(r'Beigetreten am:\s(\d{1,2}\.\s\w+\s\d{4})', content)
    # registration_date = registration_match.group(1) if registration_match else None

    # Extract number of Unterstützer*innen
    supporters_match = re.search(r'(\d+)\sUnterstützer\*in', content)
    supporters = int(supporters_match.group(1)) if supporters_match else 0

    return title, date, comments, tags, description, username, vorschlaege, konto_status, supporters

# Apply the enhanced function to the DataFrame and create new columns
df_sieburg[['Title', 'Date', 'Comments', 'Tags', 'Description', 'Username', 'Vorschläge', 'Konto Status', 'Supporters']] = df_sieburg['Content'].apply(
    lambda x: pd.Series(extract_full_data_with_supporters(x))
)




In [50]:
df_sieburg

,URL,Content,Title,Date,Comments,Tags,Description,Username,Vorschläge,Konto Status,Supporters
0,https://mitmachen.siegburg.de/proposals/1090-s...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
1,https://mitmachen.siegburg.de/proposals/1089-d...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
2,https://mitmachen.siegburg.de/proposals/1087-g...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
3,https://mitmachen.siegburg.de/proposals/1056-g...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
4,https://mitmachen.siegburg.de/proposals/1055-b...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
5,https://mitmachen.siegburg.de/proposals/1054-o...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
6,https://mitmachen.siegburg.de/proposals/1052-r...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
7,https://mitmachen.siegburg.de/proposals/1051-m...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
8,https://mitmachen.siegburg.de/proposals/1049-g...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0
9,https://mitmachen.siegburg.de/proposals/1041-e...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,20240,"60-74, 75+",Angebotslandkarte60-7475+Geselliges Beisammens...,Senioiren Cafe,1,verifiziert,0


In [73]:
df_jena["URL"][:5]


0    https://mitmachen.jena.de/matschspielplatz-schlippenstrasse
1                        https://mitmachen.jena.de/buergerbudget
2      https://mitmachen.jena.de/rahmenplanung-westbahnhofumfeld
3                     https://mitmachen.jena.de/alteschwimmhalle
4                            https://mitmachen.jena.de/jenaliebe
Name: URL, dtype: object

In [67]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import json

# Initialize the T5 model and tokenizer
model_name = "t5-small"  # You can also try "t5-base" or "t5-large" for better performance
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example raw data
raw_data = df_jena["Content"][0]

# Create a prompt for structuring the text
prompt = f"""Structure the following text into a JSON format with fields URL, Title, Date, Description, Tags, Comments, Votes, Participants, Contact, Related Links:

{raw_data}
"""

# Tokenize input
inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate structured output
outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
structured_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Attempt to parse the output as JSON
try:
    # If the model output resembles JSON, use json.loads
    structured_data = json.loads(structured_text)
    # Convert the structured data into a DataFrame
    df_jena_structured = pd.DataFrame([structured_data])
    print(df_jena_structured)  # Display the structured DataFrame
except json.JSONDecodeError:
    # Print raw output and handle malformed data
    print("Model output is not valid JSON. Here's the raw output:")
    print(structured_text)


Model output is not valid JSON. Here's the raw output:
,, Date, Description, Tags, Votes, Participants, Contact, Related Links: Matschspielplatz Schlippenstraße: In Wenigenjena, konkret auf dem Spielplatz Schlippenstraße, soll der erste Matschspielplatz entstehen. In dieser Phase können Kinder und Sorgeberechtigte ihre Ideen und Wünsche auf analogen und digitalen Wegen einbringen.PlanungsphaseAb Januar 2025 wird die Aufgabe der Spielplatzplanung an ein Planungsbüro vergeben. Diese führt gemeinsam mit der Stadtverwal


NameError: name 'e' is not defined

In [ ]:
# df['Comments'].replace(20240, 0, inplace=True)

C:\Users\Rusel\AppData\Local\Temp\ipykernel_15012\2378994937.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Comments'].replace(20240, 0, inplace=True)


In [ ]:
# Function to clean description considering keywords, numeric patterns, and refined starting logic
def clean_description_advanced(content):
    # Define keywords that mark the beginning of the description
    keywords = [
        'Geselliges Beisammensein', 'Natur', 'Hilfe & Beratung', 'Bildung', 
        'Musik', 'Bewegung', 'Glaube', 'Kulinarisches', 'Kunst & Kultur', 'Sonstiges',
    ]
    
    # Check for keywords first
    for keyword in keywords:
        if keyword in content:
            start_idx = content.find(keyword) + len(keyword)
            description = content[start_idx:].strip()
            description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
            return description

    # If no keyword is found, check for numeric patterns like "18-24, 25-49, etc."
    numeric_pattern = re.search(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    if numeric_pattern:
        start_idx = numeric_pattern.end()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # As a fallback, find the first capital letter, quote, or digit to mark the start
    fallback_match = re.search(r'[A-Z"0-9]', content)
    if fallback_match:
        start_idx = fallback_match.start()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # If nothing works, return the content as is
    return content

# Apply the advanced cleaning function to the Description column
df['Description'] = df['Content'].apply(clean_description_advanced)


In [ ]:
df

,URL,Content,Title,Date,Comments,Tags,Description,Username,Vorschläge,Konto Status,Supporters
0,https://mitmachen.siegburg.de/proposals/1090-s...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,0,"60-74, 75+",Das Cafe Lichtblick in Kaldauen lädt alle Seni...,Senioiren Cafe,1,verifiziert,0
1,https://mitmachen.siegburg.de/proposals/1089-d...,Digitalsprechstunde für Senior*innen16. Septem...,Digitalsprechstunde für Senior*innen,16. September 2024,0,"60-74, 75+",Am ersten Donnerstag im Monat erhalten Seniori...,Digitalsprechstunde für,17,verifiziert,0
2,https://mitmachen.siegburg.de/proposals/1087-g...,Gemeinschaftsgarten Cecilienstraße12. Septembe...,Gemeinschaftsgarten Cecilienstraße,12. September 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Der Gemeinschaftsgarten ist eine Kooperation d...,Gemeinschaftsgarten Cecilienstraße12,1,ist nicht verifiziert,0
3,https://mitmachen.siegburg.de/proposals/1056-g...,Geselliges Beisammensein in Braschoß18. August...,Geselliges Beisammensein in Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",in Braschoß18. August 20240 KommentareZugehöri...,Geselliges Beisammensein,3,verifiziert,4
4,https://mitmachen.siegburg.de/proposals/1055-b...,Bildungsveranstaltungen Pfarrverein Braschoß18...,Bildungsveranstaltungen Pfarrverein Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",sveranstaltungen Pfarrverein Braschoß18. Augus...,Bildungsveranstaltungen Pfarrverein,3,verifiziert,4
5,https://mitmachen.siegburg.de/proposals/1054-o...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,15. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Stöbern und Tauschen erwünscht!Den Bücherschra...,Öffentlicher Bücherschrank,3,verifiziert,2
6,https://mitmachen.siegburg.de/proposals/1052-r...,Rikscha-Fahrten in Siegburg14. August 20240 Ko...,Rikscha-Fahrten in Siegburg,14. August 2024,0,75+,Ausfahrten mit der Fahrrad Rikscha in die nähe...,Fahrten in,2,ist nicht verifiziert,4
7,https://mitmachen.siegburg.de/proposals/1051-m...,Mach was freiwillig!\r\n\r\nEhrenamt und Engag...,Mach was freiwillig!,12. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Die Freiwilligen-Agentur für den Rhein-Sieg-Kr...,Mach was,1,ist nicht verifiziert,2
8,https://mitmachen.siegburg.de/proposals/1049-g...,Generationen-Parcours08. August 20240 Kommenta...,Generationen-Parcours,08. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Die Outdoor Fitness Anlage am Michaelsberg unt...,August 20240,17,verifiziert,0
9,https://mitmachen.siegburg.de/proposals/1041-e...,Erlebe Abenteuer mit den Annopfadfinder*innen!...,Erlebe Abenteuer mit den Annopfadfinder*innen!,17. Juli 2024,0,"0-12, 13-17, 18-24",Wir sind diePfadfinder*innen vom Stamm St. Ann...,Erlebe Abenteuer,1,verifiziert,1


In [ ]:
df.to_csv('siegburg_data.csv', index=False)

In [ ]:
df['URL'][14]

'https://mitmachen.siegburg.de/proposals/1030-sommerferien-workshop-self-defense-4-girls'